# Introduction to Jupyter Notebooks
## JupyterLab 🧪

**For an overview of the JupyterLab interface**, see the **JupyterLab Welcome Tour** on this page, by going to `Help -> Welcome Tour` and following the prompts.

> **See Also**: For a more in-depth tour of JupyterLab with a full environment that runs in the cloud, see [the JupyterLab introduction on Binder](https://mybinder.org/v2/gh/jupyterlab/jupyterlab-demo/HEAD?urlpath=lab/tree/demo).


# Importations and environment

In [1]:
%pip install lsfb_dataset
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import os
from lsfb_dataset import Downloader
import cv2
from cv2 import VideoCapture
import mediapipe as mp

# Data Processing
## Download dataset
- Quote lsfb-dataset
- describing data

In [3]:
downloader = Downloader(
    dataset='isol',
    destination="../Database/Dataset",
    splits=['train','test'],
    signers=list(range(8)),
    include_cleaned_poses=True,
    include_raw_poses=False,
    include_videos=True,
    landmarks=['pose', 'left_hand', 'right_hand'],
    skip_existing_files=True,
)
downloader.download()


Metadata:   0%|          | 0/13 [00:00<?, ?files/s]

Poses for 19926 instances [pose, left_hand, right_hand]:   0%|          | 0/59778 [00:00<?, ?files/s]

Videos for 19926 instances:   0%|          | 0/19926 [00:00<?, ?files/s]

In [ ]:
# from lsfb_dataset import LSFBIsolConfig, LSFBIsolLandmarks

# dataset = LSFBIsolLandmarks(LSFBIsolConfig(
#     root="../Database/Dataset",
#     landmarks=('pose', 'left_hand', 'right_hand'),
#     sequence_max_length=50,
#     use_3d=True,
#     n_labels=10,
#     target='sign_index',
#     show_progress=True
# ))
# features, target = dataset[0]

## Select data
The selection of a video is based on the following criteria : 
 - it has more than 10 frames and less than 50 frames
 - it represents one of the ten most represented signs of the database

In [4]:
df_index = pd.read_csv('../Database/Dataset/metadata/sign_occurrences.csv')
signs = [df_index.loc[i, "sign"] for i in range(3)]
print("For the rest of the project, we will use the following words")
print(signs)

For the rest of the project, we will use the following words
['AUSSI', 'LS', 'OUI']


How many values for each word ?

In [5]:
df_instances = pd.read_csv('../Database/Dataset/instances.csv')
videos_directory = "../Database/Dataset/videos/"

def Paths_Videos_Signing(word):
    videos = []
    for i in range(len(df_instances)):
        path = videos_directory + df_instances.loc[i,"id"] + ".mp4"
        if df_instances.loc[i, "sign"]==word and os.path.exists(path):
            videos.append(path)
    return videos

for sign in signs:
    print(f"{sign} : {len(Paths_Videos_Signing(sign))}")

AUSSI : 371
LS : 342
OUI : 314


Selection of data

In [8]:
def Paths_Selected_Videos(word):
    videos = Paths_Videos_Signing(word)
    selected_videos = []
    for video in videos:
        cap = cv2.VideoCapture(video)
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if length>10 and length<50:
            selected_videos.append(video)
    return selected_videos

for sign in signs:
    print(f"Number of videos signing '{sign}' before selection : {len(Paths_Videos_Signing(sign))}")
    print(f"Number of videos signing '{sign}' after selection : {len(Paths_Selected_Videos(sign))}")

Number of videos signing 'AUSSI' before selection : 371
Number of videos signing 'AUSSI' after selection : 297
Number of videos signing 'LS' before selection : 342
Number of videos signing 'LS' after selection : 297
Number of videos signing 'OUI' before selection : 314
Number of videos signing 'OUI' after selection : 265


## Data processing

### Vectorization of data
#### Coordinates extraction with MediaPipe

> **Warning**: We are working on a particular video signing a particular sign in order to create usseful functions. If you want to see the final code vectorizing data and saving them under csv please refer too Generalisation. 


In [9]:
points = [0,4,8,12,16,20]
video = Paths_Selected_Videos("LS")[0]

- Modelisation
- Documentation MediaPipe
- Image of points on hand from MediaPipe

In [10]:
# Téléchargement du modèle détecteur de mains
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.8)

- First, 1 line for 1 point (frames in columns) :

> **Warning**: We put 0 in values lacking because we want to distinguish them from NaN (auto-completion from pandas dataframes)

In [11]:
def Vectorize(video_path, points):
    capture = VideoCapture(video_path)

    vect = [ [] for i in range(len(points))]
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            break 
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)# MediaPipe works with RGB format

        # Set flag
        image.flags.writeable = False

        #Detections
        results = hands.process(image)
        rang = 0
        for point in points:
            res = results.multi_hand_world_landmarks
            if res == None: # Without hand detection
                vect[rang].append("0")
                vect[rang].append("0")
                vect[rang].append("0")
                vect[rang].append("0")
                vect[rang].append("0")
                vect[rang].append("0")
            else :
                if len(res)>=2: # With a detection of both hands
                    hand = results.multi_hand_world_landmarks[0]
                    vect[rang].append(hand.landmark[point].x)
                    vect[rang].append(hand.landmark[point].y)
                    vect[rang].append(hand.landmark[point].z)
                    hand = results.multi_hand_world_landmarks[1]
                    vect[rang].append(hand.landmark[point].x)
                    vect[rang].append(hand.landmark[point].y)
                    vect[rang].append(hand.landmark[point].z)          
                
                elif len(res)==1: # With a one-hand detection
                    res2 = results.multi_handedness[0]
                    label = res2.classification[0].label
                    hand = results.multi_hand_world_landmarks[0]
                    if label == "Left":
                        vect[rang].append(hand.landmark[point].x)
                        vect[rang].append(hand.landmark[point].y)
                        vect[rang].append(hand.landmark[point].z)
                        vect[rang].append("0")
                        vect[rang].append("0")
                        vect[rang].append("0")
                        
                    elif label == "Right" :
                        vect[rang].append("0")
                        vect[rang].append("0")
                        vect[rang].append("0")
                        vect[rang].append(hand.landmark[point].x)
                        vect[rang].append(hand.landmark[point].y)
                        vect[rang].append(hand.landmark[point].z)
                
            rang +=1
        
    return vect

vector = Vectorize(video, points)
for point in points:
    print(f"P{point} : {vector[points.index(point)]}")

P0 : [-0.035229794681072235, 0.04214649647474289, 0.07365599274635315, 0.06615447998046875, 0.008858293294906616, 0.059337448328733444, -0.025272425264120102, 0.04661541432142258, 0.07505961507558823, 0.0662045031785965, 0.014623314142227173, 0.06222609803080559, -0.01744079403579235, 0.05195900797843933, 0.07576204836368561, 0.06387973576784134, 0.020745713263750076, 0.06564492732286453, -0.01505296304821968, 0.06329864263534546, 0.06826465576887131, 0.06825491040945053, 0.025717198848724365, 0.06839963048696518, -0.012139350175857544, 0.06553732603788376, 0.0655076801776886, 0.06601156294345856, 0.029418734833598137, 0.06589299440383911, -0.008139685727655888, 0.06643643230199814, 0.06403402239084244, 0.06316537410020828, 0.028485964983701706, 0.06471921503543854, 0.0012170583941042423, 0.062154848128557205, 0.05606139451265335, 0.05908796191215515, 0.029693743214011192, 0.06468331813812256, 0.007623791694641113, 0.06101572886109352, 0.066195547580719, 0.06099491938948631, 0.03331393

- Conversion to dataframe format

In [12]:
def Create_DataFrame(video, points):
    vect = Vectorize(video, points)
    col = ["Points"]
    
    for i in range(len(vect[0])//6):
        col += [f'xG{i}',f'yG{i}',f'zG{i}',f'xD{i}',f'yD{i}',f'zD{i}']
    
    df = pd.DataFrame(None, columns = col)
    for i in range(len(points)):
        df.loc[len(df.index)] = [f"P{4*i}"] + vect[i]

    return df

Create_DataFrame(video, points)

,Points,xG0,yG0,zG0,xD0,yD0,zD0,xG1,yG1,zG1,...,zG15,xD15,yD15,zD15,xG16,yG16,zG16,xD16,yD16,zD16
0,P0,0,0,0,0,0,0,0.060749,0.015240,0.063356,...,0.078686,0.028550,0.059636,0.075669,0.061746,0.018252,0.056754,0.029730,0.054954,0.074173
1,P4,0,0,0,0,0,0,0.001235,-0.071663,-0.006117,...,0.018136,0.061556,-0.058632,-0.000271,-0.015166,-0.006011,0.025964,0.062144,-0.047240,-0.012713
2,P8,0,0,0,0,0,0,-0.056853,-0.036342,0.015104,...,-0.031577,-0.020928,-0.071600,-0.042455,-0.034747,-0.059944,-0.013374,-0.019074,-0.061191,-0.048718
3,P12,0,0,0,0,0,0,-0.072962,-0.014099,0.023786,...,-0.020364,-0.034828,-0.036542,-0.071445,-0.056788,-0.011444,0.006156,-0.043202,-0.027421,-0.066874
4,P16,0,0,0,0,0,0,-0.064395,0.016323,0.020538,...,-0.005653,-0.002896,0.034188,-0.056776,-0.059399,0.037960,0.020592,-0.008410,0.039313,-0.038612
5,P20,0,0,0,0,0,0,-0.042759,0.055777,0.009211,...,0.002080,0.005186,0.050111,-0.033829,-0.037558,0.066984,0.019663,-0.005633,0.052318,-0.017630


#### Generalisation

- Save data from every video signing a word represented more than 100 times. --> useful for a future spread ?
- In a csv files containing all the videos signing a particular word

In [13]:
def Concatenate_DataFrame(sign, points):
    videos = Paths_Selected_Videos(sign)
    df = pd.DataFrame(None)

    if len(videos)>=100:
        for video in videos:
            df = pd.concat([df,Create_DataFrame(video, points)], ignore_index=True)
    return df
#Concatenate_DataFrame("LS", points)

In [14]:
def SaveInCSV(sign, points):
    df = Concatenate_DataFrame(sign, points)
    df.to_csv(f'../Database/Positions/{sign}.csv', index=False)

In [ ]:
for sign in signs:
    SaveInCSV(sign,points)

### Data cleaning/preparation

In [27]:
import numpy as np
from sklearn.model_selection import train_test_split

In [37]:
List_df_signs = [pd.read_csv(f"../Database/Positions/{sign}.csv") for sign in signs]
df_All = pd.concat(List_df_signs)
df_All

,Points,xG0,yG0,zG0,xD0,yD0,zD0,xG1,yG1,zG1,...,zG47,xD47,yD47,zD47,xG48,yG48,zG48,xD48,yD48,zD48
0,P0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015559,0.083049,0.044787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034177,-0.023254,-0.028474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004072,-0.096158,-0.025613,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P12,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027751,0.022401,-0.015144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021707,0.047777,-0.010973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,P4,-0.028997,-0.042513,-0.014408,0.049538,-0.024548,0.038528,-0.019001,-0.030375,-0.018096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586,P8,-0.049347,-0.006162,0.026933,0.006294,-0.057649,0.000284,-0.035912,0.003422,0.031925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1587,P12,-0.057467,0.014506,0.034316,0.021845,-0.047284,-0.020255,-0.045663,0.021027,0.035548,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1588,P16,-0.049903,0.034173,0.034643,0.029424,-0.040727,-0.034952,-0.038721,0.034162,0.029932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
All = np.array(df_All)
final_vector = []
for i in range (len(All)//len(points)):
    final_vector += [np.concatenate([All[point+i*6][1:] for point in range(6)])]
        # final_vector.append(All[num_point+i*6][1:])
print(len(final_vector))

859


In [39]:
df_All = pd.DataFrame(final_vector)
df_All

,0,1,2,3,4,5,6,7,8,9,...,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015559,0.083049,0.044787,0.006316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012938,0.082440,0.041306,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060475,0.006571,0.077605,0.012307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.049384,0.030766,0.071968,-0.016525,0.074820,0.068859,0.049453,0.028812,0.075394,-0.008487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.049849,0.021614,0.077442,-0.022396,0.072228,0.072210,0.046102,0.027305,0.079739,-0.012087,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.015995,0.019285,0.051130,0.000000,0.000000,0.000000,0.001821,0.000036,0.074452,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
855,0.033612,-0.014114,0.030079,0.007130,0.091156,-0.008275,0.040506,-0.021593,0.035359,0.023317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
856,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056637,-0.014326,0.064942,0.026982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
857,0.000000,0.000000,0.000000,0.034085,0.007794,0.072282,0.000000,0.000000,0.000000,0.061211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Add targets

In [45]:
df_correspondances = pd.read_csv('../Database/sign_to_index.csv')
targets =[]
for num_sign in range(len(signs)):
    ind = 0
    nb_videos = List_df_signs[num_sign].shape[0]//len(points)
    while ind < df_correspondances.shape[0] and df_correspondances.loc[ind, "sign"]!=signs[num_sign]:
        ind+=1
    targets += [ind]*nb_videos
print(targets)
unique, counts = np.unique(targets, return_counts=True)
dict(zip(unique, counts))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

{0: 297, 1: 297, 2: 265}

In [50]:
X = np.array(df_All)
x_train, x_test, y_train, y_test = train_test_split(X, targets, test_size=0.15)
df_training = pd.DataFrame(x_train)
df_training
df_training.insert(loc=0, column='Target', value=y_train)
df_test = pd.DataFrame(x_test)
df_test.insert(loc=0, column='Target', value=y_test)
df_training

,Target,0,1,2,3,4,5,6,7,8,...,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763
0,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000818,0.096536,0.010593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,-0.028548,0.046772,0.067884,-0.015559,0.067249,0.072289,-0.028627,0.049679,0.062371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.000000,0.000000,0.000000,0.035461,0.006311,0.085866,0.004797,0.037818,0.066588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.053047,0.030034,0.062064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0.000000,0.000000,0.000000,0.030760,0.035425,0.081729,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2,-0.022832,0.052601,0.074039,0.000000,0.000000,0.000000,-0.014985,0.061741,0.070043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007809,0.057209,0.071157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
728,0,-0.014072,0.025570,0.098022,0.000000,0.000000,0.000000,0.011060,0.022412,0.085295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_training.to_csv(f'../DataManipulation/Training.csv', index=False)
df_test.to_csv(f'../DataManipulation/Test.csv', index=False)

# Support vector machines
## Training
## Results